In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from math import ceil
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.preprocessing import QuantileTransformer
def NMAE(true, pred):
    score = np.mean(np.abs(true-pred) / true)
    return score
import sklearn
from sklearn.preprocessing import *

def temp_sensible(dataframe):
    #체감온도 공식은 체감온도(℃)=13.12+0.6215×T-11.37V0.16+0.3965V0.16×T
    #출처:https://www.sciencetimes.co.kr/news/%EC%B2%B4%EA%B0%90%EC%98%A8%EB%8F%84%EC%99%80-%EC%8B%A4%EC%A0%9C-%EC%98%A8%EB%8F%84%EB%8A%94-%EC%96%B4%EB%96%BB%EA%B2%8C-%EB%8B%A4%EB%A5%B8%EA%B0%80/
    temp_df_t=dataframe['temp_mean'].tolist()
    temp_df_w=dataframe['wind_mean'].tolist()
    temp_df_t
    temp_df_w
    result_=[]
    for i in range(len(temp_df_t)):
        result=13.12+0.6215*temp_df_t[i]-11.37*math.pow(temp_df_w[i],0.16)+0.3965*math.pow(temp_df_w[i],0.16)*temp_df_t[i]
        result_.append(round(result,1))
    dataframe.insert(4,'temp_sensible',result_)

def discomfort_index(dataframe):
    
    #0.81 * temp_df_t[i] +0.01 * temp_df_t[i]*((0.99*temp_df_t[i]) - 14.3)+ 46.3
    #불쾌지수 공식(80이상 : 매우 불쾌, 80~75: 50%불쾌 , 68~75 :불쾌감 시작, 68미만: 쾌적함)
    #discomfort_index
    temp_df_t=dataframe['temp_mean'].tolist()
    temp_df_h=dataframe['humidity'].tolist()
    temp_df_t
    temp_df_h

    result_=[]
    for i in range(len(temp_df_t)):
        result=(9/5*temp_df_t[i])-0.55*(1-temp_df_h[i]/100)*(9/5*temp_df_t[i]-26)+32
        result_.append(round(result,1))
    result_
    dataframe['discomfort_index']=result_


def pm_trafficlight(dataframe):
    pm10=dataframe['PM10'].tolist()
    pm2=dataframe['PM2.5'].tolist()
    pm_=[]
    for i in range(len(pm10)):
        if(pm10[i]<=30 or pm2[i]<=15):
            pm_.append(1)
        elif((pm10[i]<=80 and pm10[i]>30) or (pm2[i] >=16 and pm2[i]<=50)):
            pm_.append(2)
        elif((pm10[i]<=150 and pm10[i]>80)or (pm2[i]>50 and pm2[i]<=100)):
            pm_.append(3)
        elif(pm10[i]>=150 or (pm2[i]>100)):
            pm_.append(4)
    dataframe['pm']=pm_

def rental_rate(dataframe):
    y1 = dataframe[dataframe['year'] == 2018]['rental'] * 2.3
    y2 = dataframe[dataframe['year'] == 2019]['rental'] * 1.2
    y3 = dataframe[dataframe['year'] == 2020]['rental']
    new = pd.concat([y1, y2, y3], axis=0).to_frame()
    dataframe['rental'] = new['rental']
    return True




def seperate_datetime(dataframe):
    week_list = []
    year = []
    month = []
    for date in dataframe.date:
        year_point, month_point, day_point = date.split('-')
        year.append(int(year_point) - 2017)
        month.append(int(month_point))
    dataframe['year'] = year
    dataframe['month'] = month
    for day in dataframe['date']:
        num = pd.date_range(day, day, freq='D').to_series()
        week_list.append(int(num.dt.dayofweek[0]))
    dataframe['day'] = week_list
    dataframe = dataframe.drop(['date'], axis=1)
    return dataframe

def enter_week(dataframe):
    w_list = [1.02,1.02, 0.97, 0.95, 1.02, 1.01, 1.02]
    #corr 해서 상관관계에 따라 비율 설정 
    for i in range(0, 7):
        dataframe[dataframe['day'] == i] *= w_list[i]
        
clist = ['PM2.5*day_name_Sunday']


In [2]:
sample = pd.read_csv('sample_submission.csv')
sample
test_df = pd.read_csv('test.csv')
test_df
train = pd.read_csv('train.csv')

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
test_date = test_df['date']



In [3]:
date=train['date'].tolist()
cnt=0
year=[]
month=[]
day=[]
year_=''
month_=''
day_=''
for _ in date:
    list_=_.split('-')
    for i in range(3):
        if(i==0):

            year_+=list_[i]
        
        elif(i==1):
            month_+=list_[i]
        elif(i==2):
            day_+=list_[i]
            
    year.append(int(year_))
    month.append(int(month_))
    day.append(int(day_))
    
    cnt+=1
    year_=''
    month_=''
    day_=''
train_=train.drop(columns='date')
train_['year']=year
train_['month']=month
train_['day']=day

train_['precipitation']=train_['precipitation'].fillna(0)
train_['PM10'] = train_['PM10'].fillna(train_['PM10'].mean())
train_['PM2.5'] = train_['PM2.5'].fillna(train_['PM2.5'].mean())
train_ = train_.fillna(train_.mean())
rental_rate(train_)
train_.isnull().sum()

precipitation    0
temp_mean        0
temp_highest     0
temp_lowest      0
PM10             0
PM2.5            0
humidity         0
sunshine_sum     0
sunshine_rate    0
wind_mean        0
wind_max         0
rental           0
year             0
month            0
day              0
dtype: int64

In [4]:

day = []

for i in range(0,614):
    if train_.index[i] % 7 == 0:
        day.append("Monday")
    elif train_.index[i] % 7 == 1:
        day.append("Tuesday")
    elif train_.index[i] % 7 ==2:
        day.append("Wednesday")
    elif train_.index[i] % 7 == 3:
        day.append("Thursday")
    elif train_.index[i] % 7 == 4:
        day.append("Friday")
    elif train_.index[i] % 7 == 5:
        day.append("Saturday")
    elif train_.index[i] % 7 == 6:
        day.append("Sunday")
        
for i in range(0,len(train_.day)-614):
    if train_.index[i] % 7 == 0:
        day.append("Sunday")
    elif train_.index[i] % 7 == 1:
        day.append("Monday")
    elif train_.index[i] % 7 == 2:
        day.append("Tuesday")
    elif train_.index[i] % 7 == 3:
        day.append("Wednesday")
    elif train_.index[i] % 7 == 4:
        day.append("Thursday")
    elif train_.index[i] % 7 == 5:
        day.append("Friday")
    elif train_.index[i] % 7 == 6:
        day.append("Saturday")
 

train_["day_name"] = day
train_= pd.get_dummies(data = train_,columns = ['day_name'])
train_["rental"] = np.log1p(train_["rental"])
train_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   precipitation       1095 non-null   float64
 1   temp_mean           1095 non-null   float64
 2   temp_highest        1095 non-null   float64
 3   temp_lowest         1095 non-null   float64
 4   PM10                1095 non-null   float64
 5   PM2.5               1095 non-null   float64
 6   humidity            1095 non-null   float64
 7   sunshine_sum        1095 non-null   float64
 8   sunshine_rate       1095 non-null   float64
 9   wind_mean           1095 non-null   float64
 10  wind_max            1095 non-null   float64
 11  rental              1095 non-null   float64
 12  year                1095 non-null   int64  
 13  month               1095 non-null   int64  
 14  day                 1095 non-null   int64  
 15  day_name_Friday     1095 non-null   uint8  
 16  day_na

In [5]:
date=test_df['date'].tolist()
cnt=0
year=[]
month=[]
day=[]
year_=''
month_=''
day_=''
for _ in date:
    list_=_.split('-')
    for i in range(3):
        if(i==0):

            year_+=list_[i]
        
        elif(i==1):
            month_+=list_[i]
        elif(i==2):
            day_+=list_[i]
            
    year.append(int(year_))
    month.append(int(month_))
    day.append(int(day_))
    
    cnt+=1
    year_=''
    month_=''
    day_=''
test_df=test_df.drop(columns='date')
test_df['year']=year
test_df['month']=month
test_df['day']=day
test_df
test_df['precipitation']=test_df['precipitation'].fillna(0)
print(test_df.isnull().sum())


test_df=test_df.fillna(test_df.mean())

precipitation    0
temp_mean        0
temp_highest     0
temp_lowest      0
PM10             0
PM2.5            0
humidity         0
sunshine_sum     1
sunshine_rate    0
wind_mean        0
wind_max         0
year             0
month            0
day              0
dtype: int64


In [6]:
day2= []

for i in range(0,365):
    if test_df.index[i] % 7 == 0:
        day2.append("Friday")
    elif test_df.index[i] % 7 == 1:
        day2.append("Saturday")
    elif test_df.index[i] % 7 ==2:
        day2.append("Sunday")
    elif test_df.index[i] % 7 == 3:
        day2.append("Monday")
    elif test_df.index[i] % 7 == 4:
        day2.append("Tuesday")
    elif test_df.index[i] % 7 == 5:
        day2.append("Wednesday")
    elif test_df.index[i] % 7 == 6:
        day2.append("Thursday")

test_df["day_name"] = day2
test_df
test_df = pd.get_dummies(data = test_df,columns = ["day_name"])


In [7]:
discomfort_index(train_)
temp_sensible(train_)
discomfort_index(test_df)
temp_sensible(test_df)
#일교차
train_.insert(5,'temp_diff',train_['temp_highest'] - train_['temp_lowest'])
test_df.insert(5,'temp_diff',train_['temp_highest'] - train_['temp_lowest'])
#미세먼지 신호등
pm_trafficlight(train_)
pm_trafficlight(test_df)

In [8]:
train_['PM10']

0       34.0
1       36.0
2       31.0
3       39.0
4       51.0
        ... 
1090    70.0
1091    66.0
1092    69.0
1093    39.0
1094    28.0
Name: PM10, Length: 1095, dtype: float64

In [9]:
# scaler1 = QuantileTransformer()
# scaler2 = QuantileTransformer()
# # col___list = ['PM10','PM2.5','sunshine_rate','sunshine_sum']
# # col___list = ['PM10','PM2.5','sunshine_rate','sunshine_sum', 'precipitation', 'PM10','PM2.5','wind_mean','wind_max']
# col___list = ['PM10','PM2.5','sunshine_rate','sunshine_sum', 'precipitation', 'PM10','PM2.5']
# # col___list = ['PM10','PM2.5','sunshine_rate','sunshine_sum', 'precipitation', 'PM10','PM2.5','wind_mean'
# #               ,'wind_max','temp_mean','temp_highest','temp_lowest','humidity']

# # col___list = ['PM10','PM2.5','sunshine_rate','sunshine_sum', 'precipitation', 'PM10','PM2.5','wind_mean'
# #               ,'wind_max','humidity']
# # col___list = bicycle.columns
# scaler1.fit(train_[col___list])
# X_train_scaled = scaler1.transform(train_[col___list])
# train_[col___list] =X_train_scaled
# scaler2.fit(test_df[col___list])
# X_train_scaled = scaler2.transform(test_df[col___list])
# test_df[col___list] =X_train_scaled


In [10]:
# # MaxAbsScaler
# from sklearn.preprocessing import MaxAbsScaler

# col__list = ['PM10','PM2.5','sunshine_rate','sunshine_sum', 'precipitation', 'PM10','PM2.5']
# maxabs1 = MaxAbsScaler()
# maxabs2 = MaxAbsScaler()

# maxabs1.fit(train[col__list])
# x_train_scaled = maxabs1.transform(train_[col__list])
# train_[col__list] = x_train_scaled

# maxabs2.fit(test_df[col__list])
# maxabs2.fit(test_df[col__list])
# x_test_scaled = maxabs2.transform(test_df[col__list])
# test_df[col__list] = x_test_scaled

In [11]:
#Standard Scaler

from sklearn.preprocessing import StandardScaler

col__list = ['PM10','PM2.5','sunshine_rate','sunshine_sum', 'precipitation', 'PM10','PM2.5']
maxabs1 = StandardScaler()
maxabs2 = StandardScaler()

maxabs1.fit(train[col__list])
x_train_scaled = maxabs1.transform(train_[col__list])
train_[col__list] = x_train_scaled

maxabs2.fit(test_df[col__list])
maxabs2.fit(test_df[col__list])
x_test_scaled = maxabs2.transform(test_df[col__list])
test_df[col__list] = x_test_scaled

In [12]:
#미세먼지 onehot
train_ = pd.get_dummies(data = train_,columns = ['pm'])
test_df = pd.get_dummies(data = test_df,columns = ['pm'])
test_df['pm_4']=0.0
test_df['pm_4']=test_df['pm_4'].astype('uint8')
train_['dust'] = train_['PM10'] * train_['PM2.5']
train_['temp_ratio'] = abs(train_['temp_highest'] - train_['temp_lowest'])
train_['daytime'] = train_['sunshine_rate'] / train_['sunshine_sum']
train_['daytime'] = train_['daytime'].fillna(method='bfill')
test_df['dust'] = test_df['PM10'] * test_df['PM2.5']
test_df['temp_ratio'] = abs(test_df['temp_highest'] - test_df['temp_lowest'])
test_df['daytime'] = test_df['sunshine_rate'] / test_df['sunshine_sum']
test_df['daytime'] = test_df['daytime'].fillna(method='bfill')


In [13]:
test_df=test_df.fillna(0)
train_=train_.drop(columns='sunshine_sum')
test_df=test_df.drop(columns='sunshine_sum')
#train_=train_.drop(columns='sunshine_rate')
#test_df=test_df.drop(columns='sunshine_rate')
#new_row = train_.loc[[613]]
#train_ = pd.concat([train_.iloc[:613], new_row, train_.iloc[613:]], ignore_index = True)


In [14]:
train = train.drop(index=788, axis = 0)


In [15]:
import xgboost
from xgboost import XGBRegressor
from xgboost import XGBRegressor as model

y = train_['rental']
x= train_.drop('rental',axis =1)
model = XGBRegressor(objective='reg:squarederror',learning_rate=0.1,max_depth = 4, n_estimators = 1000)



model.fit(x, y)
pred = model.predict(test_df)                                   
      


XGBoostError: [22:43:48] /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/data/data.cc:1163: Check failed: valid: Input data contains `inf` or `nan`
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000011b059845 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x000000011b0b1650 unsigned long long xgboost::SparsePage::Push<xgboost::data::ArrayAdapterBatch>(xgboost::data::ArrayAdapterBatch const&, float, int) + 1296
  [bt] (2) 3   libxgboost.dylib                    0x000000011b0eae7b xgboost::data::SimpleDMatrix::SimpleDMatrix<xgboost::data::ArrayAdapter>(xgboost::data::ArrayAdapter*, float, int) + 299
  [bt] (3) 4   libxgboost.dylib                    0x000000011b0b9d25 xgboost::DMatrix* xgboost::DMatrix::Create<xgboost::data::ArrayAdapter>(xgboost::data::ArrayAdapter*, float, int, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > con

In [ ]:
                     
pred = pd.DataFrame(pred, columns=['rental'])
pred = np.expm1(pred)*1.35

In [ ]:
# # sns.pairplot(data_df, hue = 'date_year', markers = ['o','s','d'])
# # plt.show()
# sns.pairplot(train)

In [ ]:
result = pd.concat([test_date, pred],axis=1)
result = seperate_datetime(result)
enter_week(result)
# enter_month(result)
result = pd.concat([test_df, result['rental']],axis=1)
real_data = pd.read_csv('real_data.csv')
NMAE(real_data['rental'],pred['rental'])
#0.20994305332561805
#0.1759140807412157
#0.17160655151714022 result_2
#0.169358561366779
#0.16833088271950228 #1.03 예측값에 곱하면 더 오른다.
#0.1636956776772946 예측할때 곱하는거
#0.16183995947493923
# 0.16131674637594595

In [ ]:
sample['rental'] = pred
sample

In [ ]:

sample.to_csv('0.16131674637594595.csv',index=False, encoding = 'UTF-8')
real_data

In [ ]:
p1=sum(train_[train_['year']==2018]['PM10'])
p2=sum(train_[train_['year']==2019]['PM10'])
p3=sum(train_[train_['year']==2020]['PM10'])

print(p3/p1)
print(p3/p2)

In [ ]:
fu=pred[(pred['rental']>=100000)&(pred['rental']<=140000)]['rental'].index.unique()
fu

In [ ]:
# tt=[]
# for _ in fu:
#     tt.append(train_.loc[_]['PM10'])
# tt

In [ ]:
real_data[(real_data['rental']>=100000)&(real_data['rental']<=140000)]['date'].unique()



In [ ]:
p1=pred[['rental']]
p1['rental1']=real_data['rental']
p1

In [ ]:
#vkfkd색이 예측값
# 주황색이 실제값
plt.scatter(p1.index,p1['rental'])
plt.scatter(p1.index,p1['rental1'])
plt.legend()
plt.show()

In [ ]:
sum(pred['rental']*1.06)

In [ ]:
sum(real_data['rental'])

In [ ]:
plt.legend()
plt.show()